In [11]:
import pickle
from collections import defaultdict
import os
import subprocess
import yaml
from yaml import CLoader as Loader
import requests
import pandas as pd
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

(!) Нужно создать .yaml файлы с нужными годами и признаками в папке YAMLs в той же директории, где лежит этот скрипт

In [30]:
# Эта ячейка из .yaml в YAMLs, скачивает сырые .xpt в NhanesRawData и выдает .pkl в PKLs
# В PKLs только нужные признаки/параметры, но не отфильтровываются NA
-- не тестировал

def yamls2pkls(dir="YAMLs"):
    for yaml_file in os.listdir(dir):
        nhanes_from_yaml(yaml_file, f"YAMLs/{yaml_file}")

def nhanes_from_yaml(yaml_file, name: "years of nhanes period"):
    """
    ONLY ONE PERIOD FROM ONE YAML
    """
    df = pd.DataFrame()
    ddict = defaultdict()
    stream = open(yaml_file, "r")
    try:
        for doc in yaml.load_all(stream, Loader):
            for period, files in doc.items():
                for file_name, params in files.items():

                    # create dir if doesn't exist
                    if not os.path.exists("NhanesRawData"):
                        os.mkdir("NhanesRawData")
                    if not os.path.exists("PKLs"):
                        os.mkdir("PKLs")

                    # download nhanes file if not on disk
                    if not os.path.isfile(f"NhanesRawData/{file_name}.xpt"):
                        url = f"https://wwwn.cdc.gov/Nchs/Nhanes/{period}/{file_name}.XPT"
                        r = requests.get(url, allow_redirects=True)
                        open(f"NhanesRawData/{file_name}.xpt", 'wb').write(r.content)

                    # read dataframe from file, filter and rename
                    new_data = pd.read_sas(f"NhanesRawData/{file_name}.xpt", format='xport')
                    new_data = new_data[list(params["params"].keys())] # filter only needed data
                    new_data.rename(params["params"], axis=1, inplace=True)
                    if params["values"]:
                        new_data.replace(params["values"], inplace=True)

                    ddict[period] = ddict.setdefault(period, new_data).merge(new_data, how='inner')
                    df = ddict[period]

    except AttributeError:
        print("Oops! Wrong format of yaml")

    with open(f"PKLs/{name}.pkl", 'wb') as f:
        pickle.dump(df, f)

yamls2pkls()

In [31]:
# Эта ячейка чистит .plk 2019-2020 года от 2017-2018 года
-- не проверял

def nhanes_save(df, name):
    with open(f"PKLs/{name}.pkl", 'wb') as f:
        pickle.dump(df, f)

def nhanes_open(period):
    if os.path.isfile(f"PKLs/{period}.pkl"):
        with open(f"PKLs/{period}.pkl", 'rb') as f:
            df = pickle.load(f)
    else:
        raise Exception("Sorry, no nhanes for you :(")
    return df

nhanes17 = nhanes_open("17-18")
nhanes19 = nhanes_open("19-20")
nhanes19.concat(nhanes17, how="inner", ignore_index=True)

subset = list(nhanes.columns)[1::]
nhanes19_clean = nhanes19.drop_duplicates(subset=subset, keep=False)

nhanes_save(nhanes19_clean, "19-20")

,id,sex,age,race,alt,ast,creatinine,bilirubin,uric_acid,hematocrit,...,eosinophils_perc,basophils_num,basophils_perc,segmented_num,segmented_perc,hemoglobin,red_cell_cnt,rdw,mean_hemoglobin_conc,mean_hemoglobin_pg
0,109264.0,Female,13.0,Mexican American,8.0,18.0,50.39,8.55,208.2,40.5,...,1.4,5.397605e-79,0.5,2.1,46.4,13.7,4.80,13.1,33.7,28.4
1,109266.0,Female,29.0,Non-Hispanic Asian,15.0,14.0,55.69,8.55,291.5,36.5,...,0.8,5.397605e-79,0.5,4.5,58.3,12.3,4.35,14.0,33.6,28.1
2,109271.0,Male,49.0,Non-Hispanic White,8.0,14.0,68.95,8.55,297.4,44.3,...,3.8,1.000000e-01,1.0,3.0,54.8,15.3,4.54,13.7,34.4,33.5
3,109273.0,Male,36.0,Non-Hispanic White,35.0,27.0,81.33,10.26,309.3,45.5,...,1.3,5.397605e-79,0.4,3.6,58.8,15.9,5.16,12.5,34.9,30.8
4,109274.0,Male,68.0,Other Race - Including Multi-Racial,19.0,20.0,65.42,10.26,303.3,41.3,...,2.2,5.397605e-79,0.8,2.5,67.3,13.9,4.47,13.9,33.6,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10404,124815.0,Male,52.0,Non-Hispanic Black,25.0,23.0,106.08,5.13,327.1,45.4,...,2.6,5.397605e-79,0.7,2.7,50.1,15.0,5.26,14.8,33.0,28.4
10405,124817.0,Female,67.0,Mexican American,29.0,19.0,42.43,8.55,172.5,41.8,...,0.3,1.000000e-01,0.6,8.0,73.2,14.3,4.73,13.9,34.2,30.3
10406,124818.0,Male,40.0,Non-Hispanic Black,19.0,21.0,86.63,3.42,350.9,40.6,...,5.4,1.000000e-01,1.0,2.2,36.3,13.5,4.83,14.7,33.4,28.0
10407,124821.0,Male,63.0,Non-Hispanic Black,13.0,17.0,82.21,10.26,356.9,47.5,...,1.4,5.397605e-79,0.5,2.8,57.3,15.6,5.36,14.7,32.8,29.2


In [5]:
# Эта ячейка открывает все .pkl из PKLs и возвращает собранный df (dataframe)

def nhanes_concat(dir="PKLs"): # maybe not working
    nhanes_list = []
    for file in os.listdir(dir):
        if os.path.isfile(f"PKLs/{file}"):
            with open(f"PKLs/{file}", 'rb') as f:
                df = pickle.load(f)
                df["period"] = int(file[0:2])
        else:
            raise Exception("Sorry, no file")
        nhanes_list.append(df)
    nhanes = pd.concat(nhanes_list, join="inner", ignore_index=True)
    return nhanes

nhanes = nhanes_concat()
nhanes

,id,sex,age,race,alt,ast,creatinine,bilirubin,uric_acid,hematocrit,...,basophils_num,basophils_perc,segmented_num,segmented_perc,hemoglobin,red_cell_cnt,rdw,mean_hemoglobin_conc,mean_hemoglobin_pg,period
0,73557.0,Male,69.0,Non-Hispanic Black,16.0,16.0,106.96,13.68,196.3,45.4,...,1.000000e-01,1.2,2.0,42.3,15.2,5.09,14.0,33.4,29.9,13
1,73558.0,Male,54.0,Non-Hispanic White,29.0,18.0,69.84,15.39,279.6,36.7,...,1.000000e-01,0.6,7.4,58.4,11.9,3.84,13.4,32.5,31.0,13
2,73559.0,Male,72.0,Non-Hispanic White,16.0,22.0,107.85,10.26,339.0,49.9,...,1.000000e-01,0.9,4.9,68.2,17.2,5.53,13.4,34.3,31.1,13
3,73561.0,Female,73.0,Non-Hispanic White,28.0,36.0,64.53,8.55,249.8,43.8,...,1.000000e-01,1.4,4.5,68.7,14.5,4.72,12.3,33.0,30.6,13
4,73562.0,Male,56.0,Mexican American,16.0,24.0,78.68,8.55,541.3,41.5,...,1.000000e-01,0.9,6.5,69.2,14.2,4.93,13.5,34.2,28.8,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23934,124807.0,Male,44.0,Non-Hispanic Asian,16.0,16.0,79.56,8.55,368.8,44.8,...,5.397605e-79,0.3,4.0,67.2,14.5,4.79,13.6,32.5,30.3,19
23935,124810.0,Female,56.0,Non-Hispanic Black,24.0,21.0,60.11,8.55,315.2,43.6,...,5.397605e-79,0.5,6.4,74.6,14.4,5.14,14.0,33.1,28.1,19
23936,124813.0,Female,43.0,Non-Hispanic Black,13.0,14.0,59.23,6.84,273.6,38.5,...,5.397605e-79,0.9,3.0,58.0,12.6,4.77,14.3,32.8,26.4,19
23937,124814.0,Male,64.0,Non-Hispanic Black,14.0,16.0,86.63,5.13,321.2,33.7,...,2.000000e-01,2.2,4.7,57.0,10.4,4.24,19.7,30.7,24.5,19


In [16]:
# Эта ячейка строит PCA и heatmap для mean и median, сохраняет и открывает в браузере

def get_pca_components(nhanes):
    nhanes = nhanes.dropna()
    #df = shuffle(df) # чтобы точки накладывались друг на друга равномерно ТОЛЬКО ДЛЯ МНОЖЕСТВА ТОЧЕК
    features = list(nhanes.columns)[4:-1] # выделяем фичи из колонок БЕЗ ГОДА
    X = nhanes[features] # отделяем фичи
    scaler = StandardScaler() # стандартизуем
    X = scaler.fit_transform(X) # стандартизуем
    pca = PCA(n_components=2)
    components = pca.fit_transform(X)
    components = pd.DataFrame(components)
    components["period"] = nhanes["period"]
    X = pd.DataFrame(X)
    return components, X

def save_and_open_fig(fig, file="out.html"):
    """
    Saves *fig* to *file* (destination) and opens it in default .html app
    """
    if not os.path.exists("Output_figs"):
        os.mkdir("Output_figs")
    
    fig.write_html(f"Output_figs/{file}")
    try:
        os.startfile(f"Output_figs/{file}")
    except AttributeError:
        try:
            subprocess.call(['open', f"Output_figs/{file}"])
        except:
            raise Exception('Could not open file')
    
components, X = get_pca_components(nhanes)

title="Means of pca by period"
mean_period_df = components.groupby(["period"]).mean()
mean_period_df.index = mean_period_df.index.map(int).map(str)
fig = px.scatter(mean_period_df, x=0, y=1, color=mean_period_df.index, title=title)
save_and_open_fig(fig, file="mean_nhanes.html")

title="Medians of pca by period"
median_period_df = components.groupby(["period"]).median()
median_period_df.index = median_period_df.index.map(int).map(str)
fig = px.scatter(median_period_df, x=0, y=1, color=median_period_df.index, title=title)
save_and_open_fig(fig, file="median_nhanes.html")

title="Means heatmap"
features = list(nhanes.columns)[4:-1]
ddict_features = {}
for i, x in enumerate(features):
    ddict_features[i] = x
X = X.rename(columns=ddict_features)
X['period'] = nhanes['period']
mean_features = X.groupby('period').mean()
fig = px.imshow(mean_features, x=mean_features.columns, y=mean_features.index, title=title)
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = ["13", "15", "17", "19"],
        ticktext = ["13-14", "15-16", "17-18", "19-20"]
    )
)
save_and_open_fig(fig, file="mean_features_heatmap_nhanes.html")

title="Medians heatmap"
features = list(nhanes.columns)[4:-1]
ddict_features = {}
for i, x in enumerate(features):
    ddict_features[i] = x
X = X.rename(columns=ddict_features)
X['period'] = nhanes['period']
median_features = X.groupby('period').median()
fig = px.imshow(median_features, x=median_features.columns, y=median_features.index, title=title)
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = ["13", "15", "17", "19"],
        ticktext = ["13-14", "15-16", "17-18", "19-20"]
    )
)
save_and_open_fig(fig, file="median_features_heatmap_nhanes.html")